# World Internet Status

### This is an exploratory project that compares the internet speeds of various world economies. 

Economic data is in the form of per capita income from the [World Bank](http://databank.worldbank.org/data/reports.aspx?Code=NY.GDP.PCAP.CD&id=af3ce82b&report_name=Popular_indicators&populartype=series&ispopular=y#). Note that the World Bank data had gone through quite a lot of cleaning, including removing rows with non-NaN, non-numeric values, or any other values which don't play nice with Python.

The internet speeds and users datasets are downloaded from [Akamai](https://www.akamai.com/us/en/our-thinking/state-of-the-internet-report/).

### <center> I. Import Python Libraries </center></font>

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as grd
import matplotlib.ticker as tkr
import matplotlib.font_manager as font_manager

from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import FuncFormatter
from matplotlib import rcParams

#define plotter
minorLocatorx   = AutoMinorLocator(10)
minorLocatory   = AutoMinorLocator(4)
matplotlib.rc('xtick', labelsize=16) 
matplotlib.rc('ytick', labelsize=16) 
matplotlib.rcParams['axes.linewidth'] = 2.
plt.rcParams['axes.linewidth'] = 4
plt.rc('font', family='serif')
plt.rc('font', serif='Times New Roman') 
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 15
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

In [4]:
# Try to make nice displays of Pandas tables.
#Font style
from IPython.core.display import HTML
css = open('/Users/gmsardane/nikola-blog/stories/style-table.css').read() \
+ open('/Users/gmsardane/nikola-blog/stories/style-notebook.css').read()
HTML('<style>{}</style>'.format(css));

### <center> 2. Load in GDP per capita data. </center> </font>

In [5]:
## World GDP
GDP = pd.read_csv('/Users/gmsardane/datascience_project/PhilippineInternetUsers/GDP_percapita.csv')
#GDP = GDP.drop(GDP.columns[[0,1,3, -1]], axis=1)
GDP = GDP.set_index(['Country Name','Country Code']).dropna()
GDP = GDP.apply(pd.to_numeric, args=('coerce',))
GDP = GDP.reset_index()
year=[]
t=list(GDP.keys())
for word in t:
    year.append(word.split()[0])
GDP.head()

,Country Name,Country Code,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014]
0,Afghanistan,AFG,NaN,119.899037,192.153528,203.651041,224.914712,257.175795,280.245644,380.400955,384.131681,458.955782,569.940729,622.379654,690.842629,666.795051,633.569247
1,Albania,ALB,1175.788981,1326.970339,1453.642777,1890.681557,2416.588235,2709.142931,3005.012903,3603.013685,4370.539647,4114.136545,4094.358832,4437.811999,4247.485437,4411.258241,4564.390339
2,Algeria,DZA,1757.011974,1732.958517,1774.292021,2094.893302,2600.006520,3102.037384,3467.544740,3939.559939,4912.251941,3875.822095,4473.486446,5447.403976,5583.616160,5491.614414,5484.066806
3,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,ADO,21432.960070,21897.662940,24175.372750,31742.992580,37235.450030,39990.330410,42417.229150,47253.529800,46735.999570,42701.447140,39639.386020,41630.052580,39666.369210,42806.522550,NaN
